In [50]:
import pandas as pd
from typing import List
from haystack import component
import sqlite3
import gradio as gr

import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_TOKEN = os.getenv("OPENAI_API_TOKEN")

/Users/simonamazzarino/Documents/Progetti/talk_slides/2025_DevFest/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# https://www.kaggle.com/datasets/jackdaoud/esports-earnings-for-players-teams-by-game

In [20]:
df_country_and_continents_codes_list = pd.read_csv("data/country-and-continent-codes-list.csv")
df_highest_earning_players = pd.read_csv("data/highest_earning_players.csv")
df_highest_earning_teams = pd.read_csv("data/highest_earning_teams.csv")

df_highest_earning_players.head(5)

,PlayerId,NameFirst,NameLast,CurrentHandle,CountryCode,TotalUSDPrize,Game,Genre
0,3883,Peter,Rasmussen,dupreeh,dk,1822989.41,Counter-Strike: Global Offensive,First-Person Shooter
1,3679,Andreas,Højsleth,Xyp9x,dk,1799288.57,Counter-Strike: Global Offensive,First-Person Shooter
2,3885,Nicolai,Reedtz,dev1ce,dk,1787489.88,Counter-Strike: Global Offensive,First-Person Shooter
3,3672,Lukas,Rossander,gla1ve,dk,1652350.75,Counter-Strike: Global Offensive,First-Person Shooter
4,17800,Emil,Reif,Magisk,dk,1416448.64,Counter-Strike: Global Offensive,First-Person Shooter


In [21]:
connection = sqlite3.connect('database.db')

In [22]:
connection.execute('''CREATE TABLE IF NOT EXISTS country_and_continent_codes_list (
                    Continent_Name
                    Continent_Code
                    Country_Name
                    Two_Letter_Country_Code
                    Three_Letter_Country_Code
                    Country_Number);''')
connection.commit()

In [23]:
connection.execute('''CREATE TABLE IF NOT EXISTS highest_earning_players (
                    PlayerId
                    NameFirst
                    NameLast
                    CurrentHandle
                    CountryCode
                    TotalUSDPrize
                    Game
                    Genre);''')
connection.commit()

In [24]:
connection.execute('''CREATE TABLE IF NOT EXISTS highest_earning_teams (
                   TeamId
                   TeamName
                   TotalUSDPrize
                   TotalTournaments
                   Game
                   Genre
                   );''')
connection.commit()

In [25]:
print(df_country_and_continents_codes_list.to_sql('country_and_continent_codes_list', connection, if_exists='replace', index = False))
print(df_highest_earning_players.to_sql('highest_earning_players', connection, if_exists='replace', index = False))
print(df_highest_earning_teams.to_sql('highest_earning_teams', connection, if_exists='replace', index = False))

262
1000
928


In [26]:
connection.close()

In [27]:
@component
class SQLQuery:

    def __init__(self, sql_database: str):
      self.connection = sqlite3.connect(sql_database, check_same_thread=False)

    @component.output_types(results=List[str], queries=List[str])
    def run(self, queries: List[str]):
        results = []
        for query in queries:
          result = pd.read_sql(query, self.connection)
          results.append(f"{result}")
        return {"results": results, "queries": queries}

In [28]:
sql_query = SQLQuery("database.db")

In [30]:
result = sql_query.run(queries=["SELECT * FROM highest_earning_players LIMIT 10;"])
result

{'results': ['   PlayerId NameFirst   NameLast CurrentHandle CountryCode  TotalUSDPrize  \\\n0      3883     Peter  Rasmussen       dupreeh          dk     1822989.41   \n1      3679   Andreas   Højsleth         Xyp9x          dk     1799288.57   \n2      3885   Nicolai     Reedtz        dev1ce          dk     1787489.88   \n3      3672     Lukas  Rossander        gla1ve          dk     1652350.75   \n4     17800      Emil       Reif        Magisk          dk     1416448.64   \n5     16800     Jakey        Yip      Stewie2k          us     1087340.00   \n6     12183  Epitácio    de Melo          TACO          br     1063858.27   \n7     12169  Fernando  Alvarenga           fer          br     1063038.92   \n8      2455   Gabriel     Toledo        FalleN          br     1059938.92   \n9     12182   Marcelo      David      coldzera          br     1021901.46   \n\n                               Game                 Genre  \n0  Counter-Strike: Global Offensive  First-Person Shooter  \n1  

In [33]:
from haystack.tools import create_tool_from_function
def sql_query_func(queries: List[str]):
    try:
      result = sql_query.run(queries)
      return {"reply": result["results"][0]}

    except Exception as e:
      reply = f"""There was an error running the SQL Query = {queries}
              The error is {e},
              You should probably try again.
              """
      return {"reply": reply}
    
sql_tool = create_tool_from_function(sql_query_func)

In [34]:
def create_system_prompt(database_path):
    # Connect to the SQLite database
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()

    # Get all table names in the database
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [table[0] for table in cursor.fetchall()]

    # Initialize a string to hold the tables and their respective columns
    table_columns = {}

    for table in tables:
        # Read the table into a DataFrame
        df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 0", connection)
        
        # Get the column names for the current table
        columns = df.columns.to_list()
        
        # Convert column names to a comma-separated string
        columns_str = ', '.join(columns)
        
        # Store the columns for each table
        table_columns[table] = columns_str

    # Create the system prompt string with table names and respective columns
    tables_str = ', '.join(tables)
    columns_strs = ', '.join([f"table '{table}' with columns: {columns}" for table, columns in table_columns.items()])
    
    system_prompt = f"You are a helpful and knowledgeable agent who has access to an SQL database which has the following tables: {tables_str}. Each table has these respective columns: {columns_strs}. Don't reply to any other questions unrelated to the datasets."

    # Close the connection
    connection.close()

    return system_prompt

In [35]:
database_path = 'database.db'
system_prompt = create_system_prompt(database_path)
print(system_prompt)

You are a helpful and knowledgeable agent who has access to an SQL database which has the following tables: country_and_continent_codes_list, highest_earning_players, highest_earning_teams. Each table has these respective columns: table 'country_and_continent_codes_list' with columns: Continent_Name, Continent_Code, Country_Name, Two_Letter_Country_Code, Three_Letter_Country_Code, Country_Number, table 'highest_earning_players' with columns: PlayerId, NameFirst, NameLast, CurrentHandle, CountryCode, TotalUSDPrize, Game, Genre, table 'highest_earning_teams' with columns: TeamId, TeamName, TotalUSDPrize, TotalTournaments, Game, Genre. Don't reply to any other questions unrelated to the datasets.


In [ ]:
from haystack.components.agents import Agent
from haystack.utils import Secret
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage

# Create the agent with the web search tool
agent = Agent(chat_generator=OpenAIChatGenerator(model="gpt-4o-mini", api_key=Secret.from_token(OPENAI_API_TOKEN)), 
              tools=[sql_tool],
              system_prompt=system_prompt)

# Run the agent with a query
result = agent.run(messages=[ChatMessage.from_user("Which is the most valuable team in Dota 2?")],)

# Print the final response
print(result["messages"][-1].text)

The most valuable team in Dota 2 is OG, with a total prize of $33,810,636.13.


In [48]:
result

{'messages': [ChatMessage(_role=<ChatRole.SYSTEM: 'system'>, _content=[TextContent(text="You are a helpful and knowledgeable agent who has access to an SQL database which has the following tables: country_and_continent_codes_list, highest_earning_players, highest_earning_teams. Each table has these respective columns: table 'country_and_continent_codes_list' with columns: Continent_Name, Continent_Code, Country_Name, Two_Letter_Country_Code, Three_Letter_Country_Code, Country_Number, table 'highest_earning_players' with columns: PlayerId, NameFirst, NameLast, CurrentHandle, CountryCode, TotalUSDPrize, Game, Genre, table 'highest_earning_teams' with columns: TeamId, TeamName, TotalUSDPrize, TotalTournaments, Game, Genre. Don't reply to any other questions unrelated to the datasets.")], _name=None, _meta={}),
  ChatMessage(_role=<ChatRole.USER: 'user'>, _content=[TextContent(text='Which is the most valuable team in Dota 2?')], _name=None, _meta={}),
  ChatMessage(_role=<ChatRole.ASSISTAN

In [49]:
df_highest_earning_teams.groupby('TeamName')['TotalUSDPrize'].sum().reset_index().sort_values(by='TotalUSDPrize', ascending=False)


,TeamName,TotalUSDPrize
263,OG,34297886.13
392,Team Liquid,33095692.87
111,Evil Geniuses,21662171.52
461,Virtus.pro,14393878.63
250,Newbee,14072159.40
...,...,...
465,War Legend,2128.81
502,vsBANDITS,2000.00
472,Wind and Rain,1217.78
311,Rams,1200.00


In [ ]:
def chat_fn(user_input, history):
    if history is None:
        history = []

    messages = []
    for human, bot in history:
        messages.append(ChatMessage.from_user(human))
        messages.append(ChatMessage.from_assistant(bot))

    messages.append(ChatMessage.from_user(user_input))

    response = agent.run(messages=messages)
    bot_reply = response["messages"][-1].text
    
    history.append((user_input, bot_reply))
    return "", history  


with gr.Blocks() as demo:
    gr.Markdown("## 💬 Talk to Esports Data")

    chatbot = gr.Chatbot(label="Conversation", height=450)
    msg = gr.Textbox(
        placeholder="Write your message and press Enter",
        show_label=False
    )
    clear = gr.Button("🧹 Clean chat")


    msg.submit(chat_fn, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)


demo.launch(inline=True)

/var/folders/pd/vd159sk12_x73zd7gqv94kk40000gn/T/ipykernel_95337/965841644.py:23: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", height=450)


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


{'messages': [ChatMessage(_role=<ChatRole.SYSTEM: 'system'>, _content=[TextContent(text="You are a helpful and knowledgeable agent who has access to an SQL database which has the following tables: country_and_continent_codes_list, highest_earning_players, highest_earning_teams. Each table has these respective columns: table 'country_and_continent_codes_list' with columns: Continent_Name, Continent_Code, Country_Name, Two_Letter_Country_Code, Three_Letter_Country_Code, Country_Number, table 'highest_earning_players' with columns: PlayerId, NameFirst, NameLast, CurrentHandle, CountryCode, TotalUSDPrize, Game, Genre, table 'highest_earning_teams' with columns: TeamId, TeamName, TotalUSDPrize, TotalTournaments, Game, Genre. Don't reply to any other questions unrelated to the datasets.")], _name=None, _meta={}), ChatMessage(_role=<ChatRole.USER: 'user'>, _content=[TextContent(text='From which countries are the richest players in Dota 2?')], _name=None, _meta={}), ChatMessage(_role=<ChatRole